#Setup environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#---upload dataset---

import pandas as pd
train_ans = pd.read_csv('drive/My Drive/NLP_project/dataset/Training  Data/subtaskA_answers_all.csv', header=None, names=['id', 'label'], index_col='id')
train_data= pd.read_csv('drive/My Drive/NLP_project/dataset/Training  Data/subtaskA_data_all.csv', index_col='id')
dev_ans = pd.read_csv('drive/My Drive/NLP_project/dataset/Dev Data/subtaskA_gold_answers.csv', header=None, names=['id', 'label'], index_col='id')
dev_data= pd.read_csv('drive/My Drive/NLP_project/dataset/Dev Data/subtaskA_dev_data.csv', index_col='id')
test_df = pd.read_csv('drive/My Drive/NLP_project/dataset/Test Data/subtaskA_test_data.csv')

In [ ]:
#---Merge sententces with labels---

train_df = pd.concat([train_data['sent0'],train_data['sent1'],train_ans['label']],axis=1) 
dev_df = pd.concat([dev_data['sent0'],dev_data['sent1'],dev_ans['label']],axis=1)

In [ ]:
train_df

,sent0,sent1,label
id,,,
0,He poured orange juice on his cereal.,He poured milk on his cereal.,0
1,He drinks apple.,He drinks milk.,0
2,Jeff ran a mile today,"Jeff ran 100,000 miles today",1
3,A mosquito stings me,I sting a mosquito,1
4,A niece is a person.,A giraffe is a person.,1
...,...,...,...
9995,Mark ate a big bitter cherry pie,Mark ate a big sweet cherry pie,0
9996,Gloria wears a cat on her head,Gloria wears a hat on her head,0
9997,Harry went to the barbershop to have his hair cut,Harry went to the barbershop to have his glass...,1


#Simpletransforemers 

In [ ]:
pip install simpletransformers

In [ ]:
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
!sh setup.sh

In [ ]:
train_df1 = train_df.rename(columns={'sent0': 'text_a', 'sent1': 'text_b', 'label': 'labels'})
dev_df1 = dev_df.rename(columns={'sent0': 'text_a', 'sent1': 'text_b', 'label': 'labels'})
test_df1 = test_df.rename(columns={'sent0': 'text_a', 'sent1': 'text_b'})

In [ ]:
train_df1

,text_a,text_b,labels
id,,,
0,He poured orange juice on his cereal.,He poured milk on his cereal.,0
1,He drinks apple.,He drinks milk.,0
2,Jeff ran a mile today,"Jeff ran 100,000 miles today",1
3,A mosquito stings me,I sting a mosquito,1
4,A niece is a person.,A giraffe is a person.,1
...,...,...,...
9995,Mark ate a big bitter cherry pie,Mark ate a big sweet cherry pie,0
9996,Gloria wears a cat on her head,Gloria wears a hat on her head,0
9997,Harry went to the barbershop to have his hair cut,Harry went to the barbershop to have his glass...,1


In [ ]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import sklearn
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_args={
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'num_train_epochs': 8,
    "train_batch_size": 8,
    "eval_batch_size": 8,
    'save_eval_checkpoints': False,
    'save_model_every_epoch': False,
    "learning_rate": 5e-06   
    }

# Create a ClassificationModel  #bert-large-uncased-whole-word-masking
#'xlnet', 'xlnet-large-cased'
#'roberta', 'roberta-large'
model = ClassificationModel('xlnet', 'xlnet-large-cased', num_labels=2, use_cuda=True, cuda_device=0, args=train_args)
print(train_df1.head())

# Train the model
model.train_model(train_df1, eval_df=dev_df1)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(dev_df1, acc=sklearn.metrics.accuracy_score)
print(result)

In [ ]:
listdf = []
[listdf.append([A,B]) for A , B in zip(test_df1['text_a'],test_df1['text_b'])]

predictions, raw_outputs = model.predict(listdf)

In [ ]:
test_df['label'] = predictions
test_df.to_csv('drive/My Drive/NLP_project/dataset/Test Data/submission_Roberta1.csv',index=False)

# **Preprocessing**

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger') 
nltk.download('brown')
nltk.download('names')  
nltk.download('universal_tagset')  
 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [ ]:
!pip install normalise

     |████████████████████████████████| 15.7MB 195kB/s 


In [ ]:
import nltk
import string
from nltk.tokenize import word_tokenize
from normalise import normalise
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import re

wordnet_lemmatizer = WordNetLemmatizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

stop = stopwords.words('english')

punctuations = string.punctuation  #"?:!.,;"
identifiers=["the", "a", "an","The","A","An"]

def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"cause", "because", phrase)
    return phrase

def Normalization(sent):
  normalized_tokens = normalise(word_tokenize(sent), verbose=False)
  return ' '.join(normalized_tokens)


def nltk_tag_to_wordnet_tag(nltk_tag):
      if nltk_tag.startswith('J'):
          return wordnet.ADJ
      elif nltk_tag.startswith('V'):
          return wordnet.VERB
      elif nltk_tag.startswith('N'):
          return wordnet.NOUN
      elif nltk_tag.startswith('R'):
          return wordnet.ADV
      else:
          return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    #Lowercase all texts
    #remove stopwords
    
    Sent_Token=[]
    
    [Sent_Token.append(word.lower()) for word in nltk.word_tokenize(sentence) if (word not in identifiers) and (word not in punctuations)]  
    nltk_tagged = nltk.pos_tag(' '.join(Sent_Token))
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return ''.join(lemmatized_sentence)

# Lemmatizing

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.semi_supervised.label_propagation module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.semi_supervised. Anything that cannot be imported from sklearn.semi_supervised is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelPropagation from version 0.18 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
train_df= pd.DataFrame({'sent0':train_df['sent0'].apply(lambda x: lemmatize_sentence(str(x))),
                         'sent1':train_df['sent1'].apply(lambda x: lemmatize_sentence(str(x))),
                        'label':train_df['label']})

dev_df = pd.DataFrame({'sent0':dev_df['sent0'].apply(lambda x: lemmatize_sentence(str(x))),
                         'sent1':dev_df['sent1'].apply(lambda x: lemmatize_sentence(str(x))),
                        'label':dev_df['label']})
test_df = pd.DataFrame({'sent0':test_df['sent0'].apply(lambda x: lemmatize_sentence(str(x))),
                        'sent1':test_df['sent1'].apply(lambda x: lemmatize_sentence(str(x))),
                       })